In [1]:
%matplotlib inline

import sys
sys.path.append("../src")

import numpy
import scipy
import matplotlib
from matplotlib import pyplot
from astropy.io import ascii, fits

import parse
from cluster import Cluster
from rotate import apply_projection
from parse import toycluster_icfile

def p2(a):
    return ((a) * (a))

pyplot.switch_backend("module://ipykernel.pylab.backend_inline")

In [ ]:
bestfit = "/Users/Timohalbesma/Desktop/snapshot_147_010"
header, gas, dm = toycluster_icfile(bestfit)

boxsize = header["boxSize"]
boxhalf = boxsize/2

# Use Cluster instance to hold data. Toycluster parms needed for find_dm_centroid
c = Cluster(header)
c.set_header_properties()
# c.parms = parse.read_toycluster_parameterfile(glob.glob(simdir+"../ICs/*.par")[0])
c.parms = parse.read_toycluster_parameterfile(
    "/Users/Timohalbesma/Desktop/ic_both_free_cut_25.par")

# !! Domain [-boxhalf, boxhalf] for rotation matrices !!
gas["x"] -= boxhalf
gas["y"] -= boxhalf
gas["z"] -= boxhalf
dm["x"]  -= boxhalf
dm["y"]  -= boxhalf
dm["z"]  -= boxhalf

# This seems best-fit rotation angles
EulAng = numpy.array([90, 51, 45]) 
gas, dm = apply_projection(EulAng, gas, dm)

# Now find centroids in rotated image to place cygA and fidicual
# cygA at same location in plot. !! Domain find_dm_centroid [0, boxSize] !!
gas["x"] += boxhalf
gas["y"] += boxhalf
gas["z"] += boxhalf
dm["x"]  += boxhalf
dm["y"]  += boxhalf
dm["z"]  += boxhalf

In [ ]:
pyplot.figure(figsize=(12, 12))
h, xe, ye, b = pyplot.hist2d(gas["x"], gas["y"], bins=401, norm=matplotlib.colors.LogNorm())
pyplot.axhline(0, c="r")
pyplot.axhline(boxsize, c="r")
pyplot.axvline(0, c="r")
pyplot.axvline(boxsize, c="r")
#pyplot.xlim(0, boxsize)
#pyplot.ylim(0, boxsize)
pyplot.show()

In [ ]:
import os
import sys
import struct
import numpy

In [ ]:
def _write_format2_leading_block(gfile, name, size, endianness):
    '''Little helper function with speaking name, that writes the small leading
    blocks for format 2 Gadget files.'''
    gfile.write(struct.pack(endianness + ' i 4s i i', 8, name, size+8, 8))

In [ ]:
def write_header(gfile, header, endianness):
    '''
    Write header to the (Gadget-)file gfile with given format and endianness.

    Args:
        gfile (file):       The already in binary write mode opened Gadget file.
        header (dict):      The Gadget header to write.
        endianness (str):   The endianness of the file (either native '=' or
                            non-native '<' (little) or '>' (big)).
    '''

    size = 256
    
    _write_format2_leading_block(gfile, 'HEAD', size, endianness)
    gfile.write(struct.pack(endianness + 'i', size))
    start_pos = gfile.tell()
    print(start_pos)

    gfile.write(struct.pack(endianness + '6i', *header['npart']))
    gfile.write(struct.pack(endianness + '6d', *header['massarr']))
    gfile.write(struct.pack(endianness + 'd d i i', header['time'],
            header['redshift'], header['flag_sfr'], header['flag_feedback']))
    gfile.write(struct.pack(endianness + '6i', *header['npartTotal']))
    gfile.write(struct.pack(endianness + 'i i 4d 2i',
            header['flag_cooling'], header['numFiles'], header['boxSize'],
            header['omega0'], header['omegalambda'], header['hubbleParam'],
            header['flag_age'], header['flag_metals']))
    gfile.write(struct.pack(endianness + '6i', *header['numpart_total_hw']))
    gfile.write(header['la'])

    print(gfile.tell())
    assert gfile.tell() - start_pos == size
    gfile.write(struct.pack(endianness + 'i', size))

In [ ]:
with open(os.path.expanduser("GadgetWriterTest"), 'wb') as gfile:
    print(gfile)
    write_header(gfile, header, "=")

In [ ]:
header_selfwritten, gas_selfwritten, dm_selfwritten = toycluster_icfile("GadgetWriterTest", verbose=True)

In [ ]:
header

In [ ]:
header_selfwritten

In [ ]:
def write_block(gfile, block_name, data, endianness='='):
    '''
    Write a block to the (Gadget-)file gfile with given format and endianness.

    Args:
        gfile (file):       The already in binary write mode opened Gadget file.
        block_name (str):   The block name for the block to write.
        data (...):         The data to write. A UnitArr (or simplye a
                            numpy.array) for regular blocks, a header dict for
                            the HEAD block and an iterable with BlockInfo classes
                            as elements for the INFO block.
        endianness (str):   The endianness of the file (either native '=' or
                            non-native '<' (little) or '>' (big)).
    '''

    # reduce data to the numpy array
    if not isinstance(data, numpy.ndarray):
        data = numpy.array(data)

    size = data.nbytes
    _write_format2_leading_block(gfile, block_name, size, endianness)
    gfile.write(struct.pack(endianness + 'i', size))
    start_pos = gfile.tell()

    data.tofile(gfile)

    assert gfile.tell() - start_pos == size
    gfile.write(struct.pack(endianness + 'i', size))